## Что нужно сделать?

Необходимо настроить в компании репортинг ключевых показателей бизнеса таким образом, чтобы каждый сотрудник получал регулярный отчет только по тем метрикам, которые важны для его работы и доступны на его должности.

Для начала изучим, какие данные нам доступны:

In [1]:
staff = { #  словарь со списком сотрудников компании и их должностями
    'Ирина':'Customer Support',
    'Иван':'Customer Support',
    'Дмитрий':'Marketing',
    'Ольга':'CPO',
    'Анастасия':'Sales',
    'Александр':'Sales',
}

metrics = { #  словарь со списком ключевых показателей бизнеса, которые необходимы и доступны для каждой должности
    'Customer Support': ['Retention', 'MAU',],
    'Sales': ['New Users',],
    'Marketing': ['New Orders',],
    'CPO': ['Retention', 'MAU','New Users','New Orders', 'C1'], # как видите, директор по продукту следит за всеми метриками!  
    'Intern': ['Retention', 'New Users','New Orders']
}


data = { #  словарь с заранее подготовленными значениями метрик по месяцам, которые мы будем использовать в ежемесячных отчетах
    '2020-01-01':{
        'MAU':12300, # общее количество уникальных пользователей за месяц, которые воспользовались нашим сервисом
        'Retention':0.88, # доля от общего количества пользователей прошлого месяца, которая вернулась в наш сервис в текущем месяце
        'New Users':9012, # количество новых пользователей в нашем сервисе
        'New Orders': 1404 # количество новых заказов от новых пользователей нашего сервиса
    },
    '2020-02-01':{'MAU':11432,'Retention':0.75, 'New Users':12011, 'New Orders':2443 },
    '2020-03-01':{'MAU':13348,'Retention':0.89, 'New Users':8732, 'New Orders':1454 },
    '2020-04-01':{'MAU':13400,'Retention':0.93, 'New Users':9342, 'New Orders':2235 },
    '2020-05-01':{'MAU':13354,'Retention':0.79, 'New Users':11032, 'New Orders':4402 }
}

## Задание 1

Напиши функцию, которая получает дату строкой в формате `%d.%m.%Y` (например, `31.12.2020`), а возвращает строкой в формате `%Y-%m-%d` (например, `2020-12-31` – именно в таком формате указывается дата в словаре с данными метрик `data`).

In [1]:
from datetime import datetime

def modify_time(time):
    a = time.split('.')
    a.reverse()         
    return '-'.join(a)

modify_time('31.12.2020')

'2020-12-31'

## Задание 2

Теперь напиши функцию, которая получает на вход дату строкой в формате `%d.%m.%Y` и отдает все значения метрик, которые были зафиксированы в этот день, в виде словаря. Если за указанную дату показателей еще нет (то есть такой даты нет в словаре), то функция должна вернуть строку `'Нет данных'`. 

In [3]:
# Hint: тебе поможет метод dict.get(). Не забывай, что у dict.get() есть второй аргумент: значение, которое вернется, если в словаре нет искомого ключа

# написанную функцию можно проверить, передав в аргумент дату 01.02.2020

def get_metrics(date):
    a = data.get(modify_time(date))
    if a is None:
        return 'Нет данных'
    else:
        return a

get_metrics('01.02.2020') 

{'MAU': 11432, 'Retention': 0.75, 'New Users': 12011, 'New Orders': 2443}

## Задание 3

Напиши функцию, которая получает на вход имя сотрудника и отдает набор метрик из словаря `metrics`, которые необходимы человеку на этой позиции. Значения метрик находить пока не нужно — это сделаем на следующем шаге.

In [4]:
# написанную функцию можно проверить, передав строкой в аргумент имя одного из сотрудников. Например, «Ольга»

def get_metrics_for_name(name):
    position = staff.get(name)
    return metrics.get(position)

get_metrics_for_name('Ольга')
    

['Retention', 'MAU', 'New Users', 'New Orders', 'C1']

## Задание 4

Напиши функцию, которая получает на вход имя и дату и отдает все значения метрик за указанный день (тоже в виде словаря), которые необходимы человеку на его позиции.

In [5]:
# если предыдущие задания готовы, используй функции оттуда. 
# написанную функцию можно проверить, передав в аргументы имя одного из сотрудников и дату. Например, «Ольга» и «01.02.2020»

def get_metrics_for_name_and_date(name, date):
    
    time = modify_time(date) #приводим дату в норм вид
    metrics_for_date = get_metrics(time) #получаем метрики по дате
    metrics_for_name = get_metrics_for_name(name) #получаем метрики по имени
    metrics_for_return = {}
    
    for i in metrics_for_name:
        if i == 'C1':
            metrics_for_return[i] = round(metrics_for_date.get('New Orders') / metrics_for_date.get('New Users'),4)
        else:            
            metrics_for_return[i] = metrics_for_date.get(i)
    
    return metrics_for_return
          
print(get_metrics_for_name_and_date('Ольга', '01.02.2020'))
print(get_metrics_for_name_and_date('Иван', '01.03.2020'))
print(get_metrics_for_name_and_date('Дмитрий', '01.04.2020'))

{'Retention': 0.75, 'MAU': 11432, 'New Users': 12011, 'New Orders': 2443, 'C1': 0.2034}
{'Retention': 0.89, 'MAU': 13348}
{'New Orders': 2235}


## Задание 5

Теперь давай приведем выгрузку данных к красивому формату:

__«Добрый день, {имя}! Для тебя есть интересные данные за {дата}: {метрики}»__

In [6]:
name = 'Иван'
date = '01.02.2020'
metrs = list(get_metrics_for_name_and_date(name, date).keys())
message = 'Добрый день, {}! Для тебя есть интересные данные за {}: {}'.format(name,date, ', '.join(metrs)) #только метрики, без значений
#print(message)
message = 'Добрый день, {}! Для тебя есть интересные данные за {}: {}'.format(name,date,get_metrics_for_name_and_date(name, date)) #метрики со значениями
print(message)

Добрый день, Иван! Для тебя есть интересные данные за 01.02.2020: {'Retention': 0.75, 'MAU': 11432}
